# Inference of MicroRNA-Messenger RNA Interactions in TCGA-BRCA
TCGA: The Cancer Genome Atlas  
BRCA: Breast Invasive Carcinoma

# Importing Libraries and Configurations

In [1]:
import os
import sys

import pandas as pd
from joblib import Parallel, delayed
from scipy.stats import false_discovery_control, spearmanr

# Add project root to Python's path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

from config import (
    AGGREGATED_READS_FILES,
    BRCA_INTERIM_FILES_DIRS,
    BRCA_PROCESSED_FILES_DIRS,
    BRCA_PROCESSED_FILES_PATHS,
    INTERACTION_INFERENCE_PARAMETERS,
    MIRWALK_MIR_MAPPING_FILE_PATH,
    MIRWALK_PROCESSED_DATA_DIR,
)

# Function

In [2]:
def prepare_data_for_spearman(processed_dir_path):
    """
    Prepare aggregated expression data of a group for Spearman correlation analysis.

    Parameters:
    -----------
    processed_dir_path : str
        Path to the directory containing processed expression data files.

    Returns:
    --------
    dict of pd.DataFrame
        A dictionary with keys:
        - 'mir' : DataFrame with aggregated microRNA normalized read counts.
        - 'rna' : DataFrame with aggregated messenger RNA normalized read counts.
        Both DataFrames have case IDs as indices and molecule identifiers as columns.
    """
    # Create a DataFrame for the files metadata
    df_files_metadata = pd.read_csv(BRCA_PROCESSED_FILES_PATHS['files'])

    # Initialize a dictionary for Spearman-ready DataFrames
    aggregated_reads = dict()

    # Prepare the aggregated reads files
    for experimental_strategy in ['mir', 'rna']:
        # Define the experimental strategy related parameters
        drop_column = ['is_expressed']
        if experimental_strategy == 'mir':
            index_column = 'accession_id'
            file_name = AGGREGATED_READS_FILES['mir-normalized']
        else:
            index_column = 'gene_name'
            drop_column = drop_column + ['gene_id']
            file_name = AGGREGATED_READS_FILES['rna-normalized']
        
        # Get the expressed molecules and set the primary key as index
        file_path = os.path.join(processed_dir_path, file_name)
        df_agg_reads = pd.read_csv(file_path) \
            .query('is_expressed == 1') \
            .drop(columns=drop_column) \
            .set_index(keys=index_column)
        
        # Map the file ID to the associated case ID
        file_ids = list(df_agg_reads.columns)
        df_files_mapping = pd.DataFrame(file_ids, columns=['file_id'])
        df_files_mapping = df_files_mapping \
            .merge(
                right=df_files_metadata,
                left_on='file_id',
                right_on='file_id',
                how='left',
            ) \
            [['file_id', 'case_id']]
        
        # Transpose the DataFrame and set the case ID as index
        df_agg_reads = df_agg_reads \
            .transpose() \
            .reset_index() \
            .rename(columns={'index': 'file_id'}) \
            .merge(
                right=df_files_mapping,
                left_on='file_id',
                right_on='file_id',
                how='inner',
            ) \
            .drop(columns=['file_id']) \
            .set_index(keys='case_id') \
            .sort_index(ascending=True)
        
        # Store the prepared DataFrame in the dictionary
        aggregated_reads[experimental_strategy] = df_agg_reads.copy()
    
    return aggregated_reads

In [3]:
def compute_single_pair_spearman(ser_mir_expression, ser_rna_expression):
    """
    Calculate the Spearman correlation coefficient between the expression profile 
    of a single microRNA and messenger RNA pair.
    
    Parameters:
    -----------
    ser_mir_expression : pd.Series
        Expression values of a single microRNA across samples.
    ser_rna_expression : pd.Series
        Expression values of a single messenger RNA across the same samples.

    Returns:
    --------
    dict
        A dictionary containing:
        - 'accession_id': microRNA accession ID.
        - 'gene_name': gene name of the messenger RNA.
        - 'correlation': Spearman correlation coefficient.
        - 'pvalue': p-value for the hypothesis test that correlation < 0.
    """
    # Calculate the Spearman correlation coefficient for the pair
    axis = INTERACTION_INFERENCE_PARAMETERS['axis']
    alternative = INTERACTION_INFERENCE_PARAMETERS['alternative']
    correlation, pvalue = spearmanr(
        a=ser_mir_expression, # Samples of the expressed microRNA
        b=ser_rna_expression, # Samples of the expressed messenger RNAs
        axis=axis, # Each row is a observation, while the columns are variables
        alternative=alternative, # The correlation is negative in miRNA-mRNA inferred_interactions
    )
    
    # Create a dictionary to represent the results
    results = {
        'accession_id': ser_mir_expression.name,
        'gene_name': ser_rna_expression.name,
        'correlation': correlation,
        'pvalue': pvalue,
    }
    
    return results

In [4]:
def infer_mir_rna_interactions(df_mirwalk_interactions, group):
    """
    Infer significant microRNA (miRNA) and messenger RNA (mRNA) interactions based on 
    expression data and validated miRWalk interactions for a specific group of samples.

    Parameters:
    -----------
    df_mirwalk_interactions : pd.DataFrame
        A DataFrame containing known miRNA-mRNA interactions from the miRWalk database.
        Expected to contain columns ['accession_id', 'gene_name'] at minimum.
    
    group : str
        Name of the group used to locate interim and processed data directories.

    Returns:
    --------
    pd.DataFrame
        A DataFrame of statistically significant miRNA-mRNA interactions, including
        Spearman correlation coefficients, p-values, FDR-adjusted q-values, and
        miRWalk evidence.
    """
    # Define the group interim and processed directories path
    dir_base_name = (group.lower()).replace(' ', '-')
    interim_dir_path = BRCA_INTERIM_FILES_DIRS[dir_base_name]
    processed_dir_path = BRCA_PROCESSED_FILES_DIRS[dir_base_name]
    
    # Prepare the expression data of the expressed molecules
    aggregated_reads = prepare_data_for_spearman(processed_dir_path)
    df_mir_expression = aggregated_reads['mir']
    df_rna_expression = aggregated_reads['rna']
    
    # Define all potential inferred_interactions between expressed microRNAs and messenger RNAs
    df_expressed_mirs = pd.DataFrame(
        data=list(df_mir_expression.columns), columns=['accession_id']
    )
    df_expressed_rnas = pd.DataFrame(
        data=list(df_rna_expression.columns), columns=['gene_name']
    )
    df_potential_inferred_interactions = df_expressed_mirs \
        .merge(right=df_expressed_rnas, how='cross')
    
    # Select inferred_interactions with miRWalk retrieved evidence
    primary_key = ['accession_id', 'gene_name']
    df_inferred_interactions = df_potential_inferred_interactions \
        .merge(
            right=df_mirwalk_interactions,
            left_on=primary_key,
            right_on=primary_key,
            how='inner',
        )
    group_inferred_interactions = zip(
        df_inferred_interactions['accession_id'], df_inferred_interactions['gene_name'],
    )
    
    # Compute in parallel the Spearman correlation coefficient for each pair
    results = Parallel(n_jobs=-1, prefer='processes')(
        delayed(compute_single_pair_spearman)
        (df_mir_expression[mir], df_rna_expression[rna])
        for mir, rna in group_inferred_interactions
    )
    
    # Create a DataFrame for the computing results
    df_results = pd.DataFrame(results)
    
    # Adjust the p-values to control the false discovery fate (FDR)
    axis = INTERACTION_INFERENCE_PARAMETERS['axis']
    method = INTERACTION_INFERENCE_PARAMETERS['fdr-method']
    qvalues = false_discovery_control(
        ps=df_results['pvalue'], # The p-values to adjust
        axis=axis, # The axis along which to perform the adjustment
        method=method, # FDR control procedure
    )
    df_results['qvalue'] = qvalues
    
    # Add miRWalk data to results
    df_results = df_mirwalk_interactions \
        .merge(
            right=df_results,
            left_on=primary_key,
            right_on=primary_key,
            how='inner',
        )
    
    # Store the DataFrame of inferred interactions
    file_name = INTERACTION_INFERENCE_PARAMETERS['file-name']
    df_results.to_csv(os.path.join(interim_dir_path, file_name), index=False)
    
    return df_results

# miRWalk Interactions of Interest

In [5]:
# Initialize a DataFrame for the inferred interactions of interest from miRWalk
mirwalk_columns = ['mirna_name', 'gene_name', 'mirtarbase']
df_mirwalk_interactions = pd.DataFrame(columns=mirwalk_columns)

# List the interaction files downloaded from miRWalk
files = [f for f in os.listdir(MIRWALK_PROCESSED_DATA_DIR) if f.startswith('MIMAT')]

# Iterate over each inferred_interactions file from miRWalk
for file in files:
    # Create a DataFrame for the inferred interactions of interest of the microRNA
    file_path = os.path.join(MIRWALK_PROCESSED_DATA_DIR, file)    
    df_mir_inferred_interactions = pd.read_csv(file_path, low_memory=False) \
        .query('is_interaction_of_interest == 1') \
        [mirwalk_columns] \
        .drop_duplicates()
    
    # Concatenate the inferred interactions of interest to the others
    df_mirwalk_interactions = pd.concat(
        [df_mirwalk_interactions, df_mir_inferred_interactions], ignore_index=True
    )

# Add the microRNA accession IDs to the DataFrame
df_mir_mapping = pd.read_csv(MIRWALK_MIR_MAPPING_FILE_PATH)
df_mirwalk_interactions = df_mir_mapping \
    .merge(
        right=df_mirwalk_interactions,
        left_on='mirna_name',
        right_on='mirna_name',
        how='inner',
    )

In [6]:
# Print the DataFrame of inferred_interactions of interest from miRWalk
df_mirwalk_interactions

,accession_id,mirna_name,gene_name,mirtarbase
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN
...,...,...,...,...
41848,MIMAT0026483,hsa-miR-370-5p,STUM,NaN
41849,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN
41850,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN
41851,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN


# Interaction Inference

## Basal-like

In [7]:
# Infer the interactions for Basal-like tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Basal-like')

In [8]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.078607,0.234621,0.745015
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,0.018153,0.566268,0.978192
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.143526,0.907620,1.000000
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,-0.033079,0.380501,0.873011
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,0.047532,0.669012,1.000000
...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.092500,0.802930,1.000000
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.430324,0.999984,1.000000
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.221807,0.980527,1.000000
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,-0.016884,0.438326,0.911847


## HER2-enriched

In [9]:
# Infer the interactions for HER2-enriched tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'HER2-enriched')

In [10]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.013944,0.459379,0.871133
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,0.248189,0.967434,1.000000
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.003896,0.511368,0.894937
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,0.102802,0.774561,0.982053
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,-0.067943,0.309403,0.789843
...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.378153,0.997971,1.000000
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.378153,0.997971,1.000000
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.262629,0.974737,1.000000
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,0.236705,0.960494,1.000000


## Luminal A

In [11]:
# Infer the interactions for Luminal A tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Luminal A')

In [12]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.010777,0.436428,0.841128
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,-0.043499,0.259064,0.648037
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.059124,0.810222,1.000000
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,0.061249,0.818681,1.000000
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,-0.035360,0.299708,0.697952
...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.143357,0.983815,1.000000
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.456634,1.000000,1.000000
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.252987,0.999933,1.000000
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,0.017501,0.602526,0.970287


## Luminal B

In [13]:
# Infer the interactions for Luminal B tumor tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Luminal B')

In [14]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,-0.162456,0.038132,0.262916
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,0.096166,0.851953,1.000000
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.064838,0.759150,0.995618
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,0.149260,0.948140,1.000000
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,0.173825,0.971198,1.000000
...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.076256,0.796110,1.000000
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.317266,0.999793,1.000000
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.141708,0.938692,1.000000
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,-0.070633,0.221659,0.598784


## Paired Normal

In [15]:
# Infer the interactions for Paired Normal tissue
inferred_interactions = infer_mir_rna_interactions(df_mirwalk_interactions, 'Paired Normal')

In [16]:
# Print the inferred interactions associated with this group
inferred_interactions

,accession_id,mirna_name,gene_name,mirtarbase,correlation,pvalue,qvalue
0,MIMAT0000062,hsa-let-7a-5p,PKIA,NaN,0.237936,0.961289,1.000000
1,MIMAT0000062,hsa-let-7a-5p,GJC1,NaN,-0.015379,0.455214,0.898533
2,MIMAT0000062,hsa-let-7a-5p,PBX3,NaN,0.007314,0.521332,0.969059
3,MIMAT0000062,hsa-let-7a-5p,ITSN1,NaN,-0.077102,0.286103,0.674410
4,MIMAT0000062,hsa-let-7a-5p,SRGAP1,NaN,0.162748,0.884626,1.000000
...,...,...,...,...,...,...,...
41164,MIMAT0026483,hsa-miR-370-5p,STUM,NaN,0.198770,0.929030,1.000000
41165,MIMAT0026483,hsa-miR-370-5p,CRISPLD2,NaN,0.176897,0.903923,1.000000
41166,MIMAT0026483,hsa-miR-370-5p,ADAM19,NaN,0.176555,0.903485,1.000000
41167,MIMAT0026557,hsa-miR-412-5p,FAM53C,NaN,0.143814,0.854845,1.000000
